# Проект для «Викишоп»

Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

### Откроем и изучим файл

In [1]:
# загрузим библиотеки
!pip install transformers
!pip install torch
import torch
import transformers
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict

In [3]:
# прочитаем с помощью пандас наш фрейм данных
df = pd.read_csv(path)

In [4]:
# посмотрим на первые пять строк
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


*Из описания данных следует, что столбец text содержит текст комментария, а toxic — целевой признак*

In [5]:
# посмотрим на последние пять строк
df.tail()

,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


*При проведении первичного осмотра открытого фрейма данных мы можем обратить внимание, что текст содержит знаки, которые не несут никакой пользы для исследования, от них следует в дальнейшем избавиться.*

### Разбивка и создание выборок

In [6]:
# создадим целевой признак
target = df['toxic']

In [7]:
# создадим фичи
features = df['text']

In [8]:
# разделим фрейм данных на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.5, shuffle=False)

In [9]:
#проведем преобразование обучающей и тестовой выборки
corpus_train = features_train.values.astype('U')

In [10]:
corpus_test = features_test.values.astype('U')

In [11]:
corpus_train.shape

(79785,)

In [12]:
corpus_test.shape

(79786,)

*Напишим функцию clear_text(text), которая оставит в тексте только латинские символы и пробелы. На вход она принимает текст, а возвращает очищенный текст. Дополнительно уберем лишние пробелы.*

In [13]:
def clear_text(text):
    text_lem = re.sub(r'[^a-zA-Z]', ' ', text)
    return text_lem

In [14]:
# применим функцию к обучающей выборке
for i in range(len(corpus_train)):
    corpus_train[i] = clear_text(corpus_train[i])

In [15]:
# применим функцию к тестовой выборке
for i in range(len(corpus_test)):
    corpus_test[i] = clear_text(corpus_test[i])

In [16]:
# проведем «стемминг» (процесс нахождение стеммы – основы того или иного слова), ранее мы в библиотеке подгрузили EnglishStemmer
stemmer = EnglishStemmer(ignore_stopwords=False)

In [17]:
# загрузим список стоп-слов
nltk.download('stopwords')
# вызовем функцию stopwords.words(), передадим ей аргумент 'english', то есть англоязычные стоп-слова
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Резо\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# проведем рассчет TF-IDF
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(corpus_train)

In [19]:
# создадим матрицу cо значениями TF-IDF по корпусу комментариев
# на обучающей выборке
train_corp = tf_idf.transform(corpus_train)

In [20]:
# на тестовой выборке
test_corp = tf_idf.transform(corpus_test)

### Исследование моделей

*Из инструкции к проекту следует, что необходимо обучить модель классифицировать комментарии на позитивные и негативные. Необходимо построить модель со значением метрики качества F1 не меньше 0.75*

*Попробую угадать замысел поставленной задачи, скорее всего выбрана метрика F1, потому что в реальной жизни максимальная точность и полнота не достижимы одновременно и приходится искать некий баланс. Поэтому, хотелось бы иметь некую метрику которая объединяла бы в себе информацию о точности и полноте нашего алгоритма. В этом случае нам будет проще принимать решение о том какую реализацию запускать в production (у кого больше тот и круче). Именно такой метрикой является F-мера*

#### Логистическая регрессия

In [21]:
model = LogisticRegression(random_state=12345, solver = 'liblinear', class_weight='balanced')

In [22]:
model.fit(train_corp, target_train)

LogisticRegression(class_weight='balanced', random_state=12345,
                   solver='liblinear')

In [23]:
predicted = pd.Series(model.predict(test_corp))

In [24]:
# найдем F1 в модели линейная регрессия и округлим полученное значение методом round
print("Метрика F1 =", f1_score(predicted, target_test).round(2))

Метрика F1 = 0.75


#### Случайный лес

In [25]:
# исследуем работу модели с различными значениями гиперпараметра max_depth в кросс-валидации
for max_depth in range(5, 31, 5):
    rf_model = RandomForestClassifier(class_weight = 'balanced', max_depth = max_depth, n_estimators = 100)
    rf_cv = cross_val_score(rf_model, train_corp, target_train, cv=3)
    print("Score при max_depth =", max_depth, ":", rf_cv)
    print("Score mean =", sum(rf_cv)/len(rf_cv))
    print()

Score при max_depth = 5 : [0.74540327 0.64094755 0.71193833]
Score mean = 0.6994297173654195

Score при max_depth = 10 : [0.68279752 0.69817635 0.71299116]
Score mean = 0.6979883436736228

Score при max_depth = 15 : [0.74844896 0.76307577 0.71697688]
Score mean = 0.7428338660149151

Score при max_depth = 20 : [0.75521715 0.76198534 0.76540703]
Score mean = 0.7608698376887886

Score при max_depth = 25 : [0.78074826 0.78710284 0.79048693]
Score mean = 0.7861126778216457

Score при max_depth = 30 : [0.7857868  0.80966347 0.78808047]
Score mean = 0.794510246286896



*Можем обратить внимание на то, что с увеличением максимальной глубины деревьев у нас тенденция к увелчиению Score mean* 

In [26]:
# рассмотрим модель с различными значениями гиперпараметра n_estimators в кросс-валидации
for estim in range(60, 301, 60):
    rf_model = RandomForestClassifier(class_weight = 'balanced', max_depth = 5, n_estimators = estim)
    rf_cv = cross_val_score(rf_model, train_corp, target_train, cv=3)
    print("Score при n_estimators =", estim, ":", rf_cv)
    print("Score mean =", sum(rf_cv)/len(rf_cv))
    print()

Score при n_estimators = 60 : [0.63282572 0.63594661 0.69005452]
Score mean = 0.6529422823839067

Score при n_estimators = 120 : [0.69144576 0.67663095 0.71558564]
Score mean = 0.6945541141818637

Score при n_estimators = 180 : [0.68982892 0.72690355 0.67087798]
Score mean = 0.6958701510308956

Score при n_estimators = 240 : [0.66268096 0.65813123 0.67223162]
Score mean = 0.6643479350755155

Score при n_estimators = 300 : [0.66926114 0.69050573 0.67539011]
Score mean = 0.6783856614651876



In [27]:
# обучим модель RandomForestClassifier с учетом предыдущих экспериментов с гиперпараметрами
model_rfc = RandomForestClassifier(class_weight = 'balanced',
                    max_depth = 30,
                    n_estimators = 60)

In [28]:
model_rfc.fit(train_corp, target_train)

RandomForestClassifier(class_weight='balanced', max_depth=30, n_estimators=60)

In [29]:
# применим модель к тестовой выборке
predicted_rfc = pd.Series(model_rfc.predict(test_corp))

In [30]:
# применим метрику F1 для оценки нашей модели
print("Метрика F1 =", f1_score(predicted_rfc, target_test).round(2))

Метрика F1 = 0.43


**Вывод:** *метрика F1 показала очень низкий результат модели случайного леса, очевидно данная модель серьезно уступает модели линейной регрессии*

In [31]:
# классификатор на основе метода стохастического градиентного спуска (Stochastic Gradient Descent SGD)
sgd_clf = SGDClassifier(random_state=12345) 
 # обучаем классификатор
sgd_clf.fit(train_corp, target_train)
pred = cross_val_predict(sgd_clf,  train_corp, target_train, cv=3)
print("Score mean =", sum(pred)/len(pred))
#print(f1_score(pred, target_test))

Score mean = 0.04775333709343862


#### Модель линейных опорных векторов

In [33]:
model_LSVC = LinearSVC(random_state=12345, class_weight='balanced')
parameters = [{'C': [0.5, 0.4, 0.3]}]
LSVC_model_gscv = GridSearchCV(model_LSVC, param_grid=parameters, scoring='f1', n_jobs=-1, 
                   cv=5, verbose=True)
LSVC_model_gscv.fit(train_corp, target_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   13.4s finished


GridSearchCV(cv=5,
             estimator=LinearSVC(class_weight='balanced', random_state=12345),
             n_jobs=-1, param_grid=[{'C': [0.5, 0.4, 0.3]}], scoring='f1',
             verbose=True)

In [36]:
predict_lsvc = LSVC_model_gscv.best_estimator_.predict(test_corp)

In [41]:
print('Метрика F1 = {:.2f}'.format(f1_score(target_test, predict_lsvc)))

Метрика F1 = 0.76


**Вывод:** *Модель линейных опорных векторов показала наилучший результат и выходит в лидеры* 

**Общий вывод:** *В роли DS в компании интернет-магазина «Викишоп» для классифицирования комментариев на позитивные и негативные в новом сервисе мной была бы предложена модель линейных опорных векторов, которая на метрике F1 показала наилучший результат.* 